In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#!cd /content/gdrive/My\ Drive/BioImage/Project
!pip install --upgrade --force-reinstall --no-deps kaggle
import os
os.environ['KAGGLE_USERNAME'] = "serenaabraham" 
os.environ['KAGGLE_KEY'] = "0aa4e1881376dfb2149dcfb30ffe7975"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Processing /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10/kaggle-1.5.10-cp36-none-any.whl
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [ ]:
!kaggle datasets download -d andrewmvd/malignant-lymphoma-classification
!unzip -q /content/malignant-lymphoma-classification.zip

malignant-lymphoma-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from skimage import io
import matplotlib.pyplot as plt

for fn in os.listdir('inputs/MCL'):
  if(fn.endswith('.tif')):
    original = io.imread('inputs/MCL/' + fn, plugin='tifffile')
    height, width, length = original.shape
    width_cutoff=width//2
    img1=original[:,:width_cutoff,:]
    img2=original[:,width_cutoff:,:]
    plt.imsave('inputs/MCL/' + fn[:-4] + '1.png', img1)
    plt.imsave('inputs/MCL/' + fn[:-4] + '2.png', img2)
    os.remove('inputs/MCL/'+fn)

In [ ]:
!pip install mahotas
import mahotas
import os
from skimage import io
import random
import numpy as np
import time
import cv2

In [ ]:
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature
def fd_haralick(image):    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick
 
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    bins=100
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    hist.flatten()
    return hist.flatten()

In [ ]:
from sklearn.svm import SVC
from sklearn import preprocessing
def collect_features(a):
  global_features = np.hstack([fd_haralick(a), fd_hu_moments(a)])
  scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
  #Normalize The feature vectors...
  global_features=global_features.reshape(-1,1)
  rescaled_features = scaler.fit_transform(global_features)
  return rescaled_features

In [ ]:
X_train=[]
Y_train=[]
for fn in os.listdir('inputs/MCL'):
  img=io.imread('inputs/MCL/'+fn)
  X_train.append(collect_features(img))
  Y_train.append(2)

In [ ]:
X_train=np.array(X_train)
X_train=X_train[:,:,0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train,
                                                    Y_train,
                                                    test_size=.3,
                                                    random_state=1234123)

In [ ]:
import os
import random
import numpy as np
import time
!pip install mahotas
import mahotas

     |████████████████████████████████| 5.7MB 4.0MB/s 


In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


In [ ]:
from sklearn.metrics import plot_confusion_matrix
disp = plot_confusion_matrix(classifier, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=true)
disp.ax_.set_title('Normalized Confusion Matrix using SVM')

print(disp.confusion_matrix)

plt.show()